In [ ]:
import tensorflow as tf 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten


from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16

from matplotlib.pyplot import imread,imshow
from skimage.transform import rescale, resize
from skimage.io import imsave

import os
import numpy as np
from keras.models import load_model


In [ ]:
from scipy import ndimage
from skimage.filters import gaussian

def add_blur(image,sigma):
    blur_img = gaussian(image, sigma=sigma, multichannel=True)
    return blur_img

from skimage.util import random_noise
def add_gauss(image,var):
    noisy_img = random_noise(image,mode='gaussian',var=var)
    return noisy_img

def add_sp(image,amount):
    noisy_img = random_noise(image,mode='s&p',amount=amount)
    return noisy_img

from skimage.transform import AffineTransform,warp
def add_shift(image,dim):
    #dim is a tuple to indicate by which values to shift 
    image[0,:]=0.0
    image[:,0]=0.0
    transform = AffineTransform(translation=(dim[0],dim[1]))
    shifted = warp(image, transform, mode='edge')
    return shifted

def add_occlusion(image,quarter):
    #quarter is a number from 0 to 3 to indicate which quarter to hide
    image = np.array(image)
    image[:, quarter*int(1*image.shape[1]/4): (quarter+1)*int(1*image.shape[1]/4) ] = np.max(image)
    return image 

from skimage import exposure
def change_gamma(image,gamma):
    image_bright = exposure.adjust_gamma(image, gamma=gamma,gain=1)
    return image_bright

def add_noise(image,param,noise_type):
    if noise_type == 'gamma':
        noisy_img = change_gamma(image,param)
    elif noise_type == 'occlusion':
        noisy_img = add_occlusion(image,param)
    elif noise_type == 'shift':
        noisy_img = add_shift(image,param)
    elif noise_type == 'gauss':
        noisy_img = add_gauss(image,param)
    elif noise_type == 'sp':
        noisy_img = add_sp(image,param)
    elif noise_type == 'blur':
        noisy_img = add_blur(image,param)
    return noisy_img

In [ ]:
# Dataset parameters 
dataset_path = '../../datasets/VisualPhish/'
reshape_size = [224,224,3]

# Saved model
output_dir = './'
model_name = 'model.h5'
phish_train_idx_name = 'train_idx.npy'
phish_test_idx_name = 'test_idx.npy'

#noise type
noise = 'gamma'
parameter = 0.7

# Load Phishing data with noise

In [ ]:

def read_imgs_per_website(data_path,targets,imgs_num,reshape_size,start_target_count):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    
    all_file_names = []
    targets_list = targets.splitlines()
    count = 0
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        print(target_path)
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            try:
                img = imread(target_path+'/'+file_names[j])
                img = img[:,:,0:3]
                img = add_noise(img,parameter,noise)
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                all_labels[count,:] = i + start_target_count
                all_file_names.append(file_names[j])
                count = count + 1
            except:
                #some images were saved with a wrong extensions 
                try:
                    img = imread(target_path+'/'+file_names[j],format='jpeg')
                    img = img[:,:,0:3]
                    img = add_noise(img,parameter,noise)
                    all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                    all_labels[count,:] = i + start_target_count
                    all_file_names.append(file_names[j])
                    count = count + 1
                except:
                    print('failed at:')
                    print('***')
                    print(file_names[j])
                    break 
    return all_imgs,all_labels,all_file_names


In [ ]:
# Read images phishing
data_path = dataset_path + 'phishing/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 1195
all_imgs_test,all_labels_test,all_file_names_test = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

phish_test_idx = np.load(output_dir+phish_test_idx_name)
phish_train_idx = np.load(output_dir+phish_train_idx_name)

# Apply noise to test only 
X_phish_test = all_imgs_test[phish_test_idx,:]

# Load model and predict

In [ ]:
# Load_model 
margin = 2.2
def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value

final_model = load_model(model_name, custom_objects={'loss': loss})
inside_model = final_model.layers[3]

X_phish_test_noise = inside_model.predict(X_phish_test)

np.save('X_phish_test_noise_'+noise,X_phish_test_noise)